<a href="https://colab.research.google.com/github/slawekk1717/finance101/blob/main/Finance101/finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# pd.read_csv('data/example.csv')
# pd.read_csv('https://query1.finance.yahoo.com/v7/finance/download/...')

In [ ]:
import yfinance as yf

def load_prices(tickers, interval='1mo'):
    data = yf.download(tickers=tickers, interval=interval, auto_adjust=False, progress=False)
    try:
        prices = data['Adj Close']
    except KeyError:
        prices = data['Close']
    prices = prices.copy()
    if not isinstance(prices.index, pd.DatetimeIndex):
        prices.index = pd.to_datetime(prices.index)
    return prices.squeeze()

rets = load_prices('SPY')


In [ ]:
type(rets) # 1 dimensional pd.Series for single stock

In [ ]:
rets

In [ ]:
rets = load_prices(['SPY', 'BND'])


In [ ]:
type(rets) # 2 dimensional pd.DataFrame for multiple stocks

In [ ]:
rets

In [ ]:
rets.dropna(inplace=True)
rets

In [ ]:
rets = rets.to_period('M')
rets # Price Series

In [ ]:
rets.plot()

### Single Period Returns

Return from time period $i$ (initial) to $f$ (final):

$$ R_{i, f} = \frac{P_{f}-P_{i}}{P_{i}} $$

Buy at 10, sell at 13

$$ R_{10, 13} = \frac{P_{13}-P_{10}}{P_{10}} = \frac{3}{10} = 0.3 = 30\% $$

Identical to:

$$ R_{i, f} = \frac{P_{f}}{P_{i}} - 1 $$

$$ R_{10, 13} = \frac{13}{10} - 1 = 1.3 - 1 = 0.3 = 30\% $$

Price Return if ( $P_{f}$ = Price )

Total Return if ( $P_{f}$ = Price + Cashflows)

### Multi Period Returns

$$ R_{t_1, t_3} = (1 + R_{t_1, t_2})(1 + R_{t_2, t_3}) - 1$$

DAY 1: 30% gain

DAY 2: 30% loss

$$ R_{t_1, t_3} = (1 + 0.3)(1 + -0.3) - 1 = (1.3)(0.7) - 1  = 0.91 - 1 = -.09  = -9\%$$

Compound (geometric) returns are NOT additive

Geometric return = -9%

Arithmetic return = 30 + (-30) = 0%

In [ ]:
# Returns cannot be computed for the first day, as previous closing prices are not available
# Whenever we convert from prices to returns, we lose a single data point

In [ ]:
# Convert prices to returns
rets = rets.pct_change().dropna() # Return Series

In [ ]:
rets.plot()

In [ ]:
compound_returns = (rets + 1).prod() - 1
(compound_returns * 100).round(2).astype('str') + '%'

In [ ]:
rets.head()
rets.tail()
rets.size
rets.shape

In [ ]:
rets.index
rets.columns
rets['SPY']
rets[['SPY']]

In [ ]:
rets.loc['2009-02']
rets.iloc[20]

In [ ]:
rets.loc['2009-02': '2009-05']
rets.iloc[20:24]

### Measures of Risk

Variance
$$ \sigma^2 = \frac{1}{N} {\sum_{i=1}^{N}(R_i - \mu)^2} $$

Standard Deviation - Square Root of Variance

In [ ]:
rets.std() # Standard Deviation

Annualizing Returns

If you have a return of 1% / month, what is the annualized return?

$$ R_{annualized} = ( (1 + 0.01) (1 + 0.01) (1 + 0.01) ... (1 + 0.01) ) - 1 $$

$$ R_{annualized} = (1 + 0.01)^{12} - 1 $$

In [ ]:
n_periods = rets.shape[0]
compounded_growth = (1+rets).prod()
monthly_ret = compounded_growth**(1/n_periods) - 1
(monthly_ret + 1)**12 - 1

In [ ]:
def annualize_rets(r, periods_per_year=12):
    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year / n_periods) - 1

In [ ]:
annualize_rets(rets)

Annualized Standard Deviation
$$ \sigma * \sqrt{p}$$

In [ ]:
def annualize_vol(r, periods_per_year=12):
    return r.std() * (periods_per_year**0.5)

In [ ]:
annualize_vol(rets)

Sharpe Ratio

In [ ]:
# Raw Sharpe Ratio
annualize_rets(rets) / annualize_vol(rets)

In [ ]:
# Growth of 1 dollar
wealth_index = (1+rets).cumprod()
wealth_index.plot()

In [ ]:
rets = load_prices('SPY').pct_change().dropna()

In [ ]:
wealth_index = (1+rets).cumprod()
start_date = wealth_index.index.min() - pd.DateOffset(months=1)
wealth_index = pd.concat([pd.Series([1], index=[start_date]), wealth_index])

previous_peaks = wealth_index.cummax()
drawdowns = (wealth_index - previous_peaks)/previous_peaks

plt.figure(figsize=(12, 6))
max_drawdown = drawdowns.min()
max_drawdown_date = drawdowns.idxmin()

plt.plot(wealth_index)
plt.plot(previous_peaks)
plt.plot(drawdowns)

plt.annotate(f'Max Drawdown: {max_drawdown:.2%}', xy=(max_drawdown_date, max_drawdown),
            xytext=(max_drawdown_date + pd.DateOffset(years=2), max_drawdown * 1.2),
            arrowprops=dict(arrowstyle='->', lw=1), color='blue')